In [15]:
import torch
import numpy as np
import pandas as pd
import random 
from Evaluator import ModelEvaluator
from sklearn.model_selection import train_test_split 

## cargando los datos adversarios 2021

In [16]:
advData=pd.read_csv('datasets/Adversarial_data.csv')
print(advData.head(3))


   -97.0  -47.0  -6.0  -27.0  73.0  13.0  -59.0  93.0  -36.0   5.0
0   -1.0  -65.0 -26.0   51.0  -9.0  23.0   63.0 -54.0   19.0  15.0
1   -1.0   63.0 -93.0   43.0 -12.0  76.0   -4.0 -53.0   62.0  50.0
2   53.0  -25.0  42.0   87.0   1.0   1.0   18.0  -6.0   83.0  11.0


In [17]:
advData.columns=['tcp_packets','dist_port_tcp','external_ips','vulume_bytes','udp_packets','source_app_packets','remote_app_packets','source_app_bytes','remote_app_bytes','dns_query_times']

Agregando los headers, a partir del conocimiento de los datos originales, teniendo en cuenta que el entrenamiento no es un proceso que realiza el atacante.

In [18]:
advData.head(3)

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times
0,-1.0,-65.0,-26.0,51.0,-9.0,23.0,63.0,-54.0,19.0,15.0
1,-1.0,63.0,-93.0,43.0,-12.0,76.0,-4.0,-53.0,62.0,50.0
2,53.0,-25.0,42.0,87.0,1.0,1.0,18.0,-6.0,83.0,11.0


In [19]:
advData['type']='malicious'

In [20]:
advData.head(3)

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times,type
0,-1.0,-65.0,-26.0,51.0,-9.0,23.0,63.0,-54.0,19.0,15.0,malicious
1,-1.0,63.0,-93.0,43.0,-12.0,76.0,-4.0,-53.0,62.0,50.0,malicious
2,53.0,-25.0,42.0,87.0,1.0,1.0,18.0,-6.0,83.0,11.0,malicious


## Cargando los datos adversarios 2018

In [21]:
dataAdversarial2018=pd.read_csv("datasets/Colcom2018AdversarialDataTrain.csv",header=0,sep=';')
print(dataAdversarial2018.head(3))
print(dataAdversarial2018.shape)

   tcp_packets  dist_port_tcp  external_ips  vulume_bytes  udp_packets  \
0     5.927622            0.0     -0.666667     39.884274          0.0   
1    22.138217            0.0     -0.666667     -0.312383          0.0   
2   104.151100            0.0     -0.666667    107.834553          0.0   

   source_app_packets  remote_app_packets   source_app_bytes  \
0          103.199390            0.700097         437.493903   
1           31.467176           -0.258824         625.873298   
2           35.351266           -0.270588         326.570370   

   remote_app_bytes  dns_query_times       type  
0          1.994524            -0.25  malicious  
1         24.786021            -0.25  malicious  
2         60.857192            -0.50  malicious  
(500, 11)


## Cargando los datos originales

In [22]:
data=pd.read_csv("datasets/Colcom2018CleanedScaled.csv",header=0,sep=';')
print(data.head(3))
print(data.shape)

   tcp_packets  dist_port_tcp  external_ips  vulume_bytes  udp_packets  \
0     0.127907            6.0      0.333333      0.036782          0.0   
1     1.069767            0.0      2.333333      1.786894          0.0   
2     1.988372            0.0      1.333333      1.843764          0.0   

   source_app_packets  remote_app_packets   source_app_bytes  \
0            0.100000            0.105882           0.041652   
1            1.088889            0.976471           0.897275   
2            1.944444            2.235294           6.465984   

   remote_app_bytes  dns_query_times    type  
0          0.029865              0.0  benign  
1          1.787684              2.0  benign  
2          1.831939              1.5  benign  
(7832, 11)


# Split for training and testing

## Primero los datos originales

In [23]:
golden_X_train, golden_X_test, golden_Y_train, golden_Y_test = train_test_split(data.iloc[:,0:10], data['type'], test_size=0.25, random_state=50)

## Segundo los datos del 2018

In [24]:
oldAdv_X_train, oldAdv_X_test, oldAdv_Y_train, oldAdv_Y_test = train_test_split(dataAdversarial2018.iloc[:,0:10], dataAdversarial2018['type'], test_size=0.25, random_state=50)

## Ahora los datos adversarios

In [25]:
adv_X_train, adv_X_test, adv_Y_train, adv_Y_test = train_test_split(advData.iloc[:,0:10], advData['type'], test_size=0.25, random_state=50)

## Mezclando los datos para obtener un modelo mas robusto

In [32]:
golden_X_train.iloc[0:3,0:10]

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times
2883,0.267442,7.0,0.666667,0.260066,0.0,0.244444,0.247059,0.076528,0.254787,0.25
2926,1.593023,2.0,1.000000,0.988394,0.0,1.511111,1.670588,7.028402,0.963462,0.25
843,1.023256,109.0,0.000000,1.026158,0.0,0.933333,1.035294,0.101653,0.979634,-0.50


In [37]:
golden_X_train=golden_X_train.reset_index(drop=True)
golden_Y_train=golden_Y_train.reset_index(drop=True)
golden_X_train.iloc[0:3,:]

,tcp_packets,dist_port_tcp,external_ips,vulume_bytes,udp_packets,source_app_packets,remote_app_packets,source_app_bytes,remote_app_bytes,dns_query_times
0,0.267442,7.0,0.666667,0.260066,0.0,0.244444,0.247059,0.076528,0.254787,0.25
1,1.593023,2.0,1.000000,0.988394,0.0,1.511111,1.670588,7.028402,0.963462,0.25
2,1.023256,109.0,0.000000,1.026158,0.0,0.933333,1.035294,0.101653,0.979634,-0.50


## Agregando a los datos originales los datos del 2018

In [ ]:
golden_X_train=golden_X_train.append(oldAdv_X_train, ignore_index = True)
golden_Y_train=golden_Y_train.append(oldAdv_Y_train, ignore_index = True)

## Agregando a los datos originales los datos adversarios del 2021

In [ ]:
golden_X_train=golden_X_train.append(adv_X_train, ignore_index = True)
golden_Y_train=golden_Y_train.append(adv_Y_train, ignore_index = True)

## Load discriminator

In [27]:
eval=ModelEvaluator()
eval.test(AdvData[:1])

NameError: name 'AdvData' is not defined

## Secure learning